In [12]:
from polygon import RESTClient
# import polygon
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime, timedelta, date
import time

from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
    USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
    USLaborDay, USThanksgivingDay

client = RESTClient(api_key="LWTCy8ovzCTqIq4fdBxFmoTptFxZ2pAo")

## INTRADAY-DATA UNDERLYING

In [29]:
## File path where EOD data is present and you want to append today's data to.
df = pd.read_csv(r"C:\Data\USData\Underlying\1min\SPY.csv")
ticker = "SPY"

df['Date'] = pd.to_datetime(df['Date'],format='mixed',dayfirst=True)
## for daily updation
end_date = date.today()
current_day,current_month,current_year = end_date.strftime('%d'),end_date.strftime("%m"),end_date.strftime('%Y')
end_date_format = current_year+'-'+current_month+'-'+current_day
start_date_format = current_year+'-'+current_month+'-'+str(int(current_day)-1)

## in case you want data for a range of dates
# start_date_format = '2023-10-23'
# end_date_format = '2023-10-25'

## timespan = Day | minute | hours 

aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_=start_date_format, to=end_date_format, limit=50000):
    aggs.append(a)

aggsDataframe = pd.DataFrame(aggs)
aggsDataframe['Date'] = pd.to_datetime(aggsDataframe['timestamp'],unit='ms')
aggsDataframe['Time'] = aggsDataframe['Date'].dt.time
aggsDataframe['Date'] = aggsDataframe['Date'].dt.date
aggsDataframe['Ticker'] = ticker + '.EQ'
aggsDataframe = aggsDataframe[['Ticker','Date','Time','open','high','low','close','volume','vwap','transactions','otc']]
final_df = pd.concat([df,aggsDataframe],ignore_index=True)
final_df.drop_duplicates(inplace=True)

date_list = aggsDataframe['Date'].unique()
inception_date_list = df['Date'].dt.date.unique()

if any(x in date_list for x in inception_date_list):
    print("Dates are already present in the main file")
    print("Break")

else:
    if final_df.shape[0] == df.shape[0] + aggsDataframe.shape[0]:
        print("Sanity Check success")
        aggsDataframe.to_csv(r"C:\Data\USData\Underlying\1min\SPY.csv",mode='a', header = not os.path.exists(r"C:\Data\USData\Underlying\1min\SPY.csv"),index=False)
    else:
        print("Error")
        print("Appended 1min data till latest available.")

Dates are already present in the main file
Break


## INTRADAY-DATA UNDERLYING

In [27]:
## File path where EOD data is present and you want to append today's data to.
df = pd.read_csv(r"C:\Data\USData\Underlying\1min\SPY.csv")
ticker = "SPY"

df['Date'] = pd.to_datetime(df['Date'],format='mixed',dayfirst=True)

## for daily updation
end_date = date.today()
current_day,current_month,current_year = end_date.strftime('%d'),end_date.strftime("%m"),end_date.strftime('%Y')
end_date_format = current_year+'-'+current_month+'-'+current_day
start_date_format = current_year+'-'+current_month+'-'+str(int(current_day)-1)

## in case you want data for a range of dates
# start_date_format = '2023-10-23'
# end_date_format = '2023-10-25'

## timespan = Day | minute | hours 

aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_=start_date_format, to=end_date_format, limit=50000):
    aggs.append(a)

aggsDataframe = pd.DataFrame(aggs)
aggsDataframe['Date'] = pd.to_datetime(aggsDataframe['timestamp'],unit='ms')
aggsDataframe['Time'] = aggsDataframe['Date'].dt.time
aggsDataframe['Date'] = aggsDataframe['Date'].dt.date
aggsDataframe['Ticker'] = ticker + '.EQ'
aggsDataframe = aggsDataframe[['Ticker','Date','Time','open','high','low','close','volume','vwap','transactions','otc']]
final_df = pd.concat([df,aggsDataframe],ignore_index=True)
final_df.drop_duplicates(inplace=True)

date_list = aggsDataframe['Date'].unique()
inception_date_list = df['Date'].dt.date.unique()

if any(x in date_list for x in inception_date_list):
    print("Dates are already present in the main file")
    print("Break")
else:
    if final_df.shape[0] == df.shape[0] + aggsDataframe.shape[0]:
        print("Sanity Check success")
        aggsDataframe.to_csv(r"C:\Data\USData\Underlying\1min\SPY.csv",mode='a', header = not os.path.exists(r"C:\Data\USData\Underlying\1min\SPY.csv"),index=False)
    else:
        print("Error")
        
    print("Appended 1min data till latest available.")

Dates are already present in the main file
Break


## GETTING DAILY EXPIRY WISE CONTRACTS

In [10]:
## NOTE - The date entered here, will basically give data for the previous day and not the data given as an input

end_date = date.today()
current_day,current_month,current_year = end_date.strftime('%d'),end_date.strftime("%m"),end_date.strftime('%Y')
end_date_format = current_year+'-'+current_month+'-'+current_day
start_date_format = current_year+'-'+current_month+'-'+str(int(current_day)-1)

def getting_rawdata():
    contract_names = []
    for c in tqdm(client.list_options_contracts(underlying_ticker = 'SPY',expiration_date_gte=start_date_format,expiration_date_lt=end_date_format,limit=1000,expired='true')):
        contract_names.append(c)
    # contract_names
    
    final_df = pd.DataFrame()
    for i in tqdm(contract_names):
        try:
            dailyOptionData = client.get_aggs(ticker=i.ticker,
                                             multiplier=1,
                                             timespan='minute',
                                             from_='2021-01-01',
                                             to=end_date_format,
                                             limit=50000)
            optionDataFrame = pd.DataFrame(dailyOptionData)
            if optionDataFrame.empty == True:
                continue
            else:
                optionDataFrame['Ticker'] = i.ticker
                final_df = pd.concat([final_df,optionDataFrame],ignore_index=True)
        except:
            pass
            
    final_df['Date'] = pd.to_datetime(final_df['timestamp'],unit='ms')
    final_df['Time'] = final_df['Date'].dt.time
    final_df['Date'] = final_df['Date'].dt.date
    final_df = final_df.sort_values(by=['Date'])
    return final_df


def holiday_list():
    class USTradingCalendar(AbstractHolidayCalendar):
        rules = [
            Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
            USMartinLutherKingJr,
            USPresidentsDay,
            GoodFriday,
            USMemorialDay,
            Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
            USLaborDay,
            USThanksgivingDay,
            Holiday('Christmas', month=12, day=25, observance=nearest_workday),
            Holiday('JuneteenthNationalIndependenceDay', month=6, day=19, observance=nearest_workday)
        ]
    
    
    def get_trading_close_holidays(year):
        inst = USTradingCalendar()
    
        return inst.holidays(datetime(year-1, 12, 31), datetime(year, 12, 31))
    
    
    if __name__ == '__main__':
        holiday_list = []
        year_range = int(int(current_year) - 2)
        for i in range(3):
            holiday_list.append(get_trading_close_holidays(year_range).date)
            year_range+=1
    
    new_list = []
    for i in range(len(holiday_list[:])):
        for j in range(len(holiday_list[i][:])):
            new_list.append(holiday_list[i][j])
    new_list = [date_obj.strftime('%Y%m%d') for date_obj in new_list]
    holiday_set = sorted(list(set(new_list)))
    # print(holiday_set)
    return holiday_set

def contract_creation():
    options_data = getting_rawdata()
    # options_data.to_csv(rf'C:\users\admin\desktop\SPY_{end_date}.csv',index=False)
    
    # Sort the options data by date and time
    options_data['Date'] = pd.to_datetime(options_data['Date'])
    options_data['Expiry_Dates'] = pd.to_datetime(options_data['Ticker'].str[5:11],format='%y%m%d')
    options_data['Strike'] = options_data['Ticker'].str[-8:-3]
    # options_data = options_data.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])
    
    # Create a list of unique expiration dates
    expiration_dates = options_data['Expiry_Dates'].unique()
    
    # Initialize a DataFrame to store the daily continuous contracts
    daily_continuous_contracts = []
    
    # Iterate through each trading day
    for trading_day in tqdm(pd.date_range(start=options_data['Date'].min(), end=options_data['Date'].max(), freq='B')):
        # Filter options data for the current trading day
        daily_data = options_data[(options_data['Date'].dt.date == trading_day.date())]
    
        for expiration_date in expiration_dates:
            contract_data = daily_data[daily_data['Expiry_Dates'] == expiration_date]
            start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
            if contract_data.empty == True:
                continue
            start,end = contract_data['Date'].dt.date.unique(),contract_data['Expiry_Dates'].dt.date.unique()
            start = [date_obj.strftime('%Y-%m-%d') for date_obj in start]
            end = [date_obj.strftime('%Y-%m-%d') for date_obj in end]
            dte = np.busday_count(start,end,holidays=holiday_set)
            # print(expiration_date,start,end,dte)
            contract_data['DTE'] = dte[0]
            
            daily_continuous_contracts.append(contract_data)
    
    # Concatenate all daily continuous contracts into a single DataFrame
    daily_continuous_contract = pd.concat(daily_continuous_contracts)
    
    # Sort the continuous contract by date and time
    daily_continuous_contract = daily_continuous_contract.sort_values(by=['Date', 'Expiry_Dates', 'Strike'])
    
    # Save the daily continuous contract to a CSV file
    # daily_continuous_contract.to_csv(r"C:\Data\USData\Continuous_Contracts\AllContracts\\Daily_continuous_SPY_options_data.csv", index=False)
    
    print("Creating continuous contracts and labelling.")
    weekly_groups = daily_continuous_contract.groupby('DTE')
    unique_values = list(daily_continuous_contract['DTE'].unique())
    unique_values = sorted([a for a in unique_values if a>=0 and a<=5])
    for i in sorted(unique_values):
        print(f'DTE {i}')
        dte_df = weekly_groups.get_group(i).sort_values(by=['Date']).reset_index(drop=True)
        dte_df['New_Ticker'] = dte_df['Ticker'].str[:5] + f'D{i}' + dte_df['Ticker'].str[11:]
        # display(dte_df)
        # dte_df.to_csv(rf"C:\Data\USData\Continuous_Contracts\DailyContracts\DTE{i}.csv",index=False)
        dte_df.to_csv(rf"C:\users\admin\desktop\DTE{i}.csv",index=False)
    print("Daily continuous contracts created and saved.")

st = time.time()
holiday_set = holiday_list()
contract_creation()
et = time.time()
print("Total time taken",f'{(et-st):0,.2f} seconds.')

0it [00:00, ?it/s]

  0%|          | 0/280 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

Creating continuous contracts and labelling.
DTE 0
DTE 1
DTE 3
DTE 4
DTE 5
Daily continuous contracts created and saved.
Total time taken 175.18
